#  Submit geo opt for molecules, slabs and bulks

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
# AiiDA imports.
%aiida

# General imports.
import ipywidgets as ipw
from IPython.display import clear_output
from traitlets import dlink


# AiiDAlab imports.
import aiidalab_widgets_base as awb

# Custom imports.
from widgets.build_slab import BuildSlab
from widgets.inputs import InputDetails
from widgets.empa_viewer import EmpaStructureViewer
from widgets.import_cdxml import CdxmlUpload2GnrWidget

In [ ]:
Cp2kMoleculeOptWorkChain = WorkflowFactory('nanotech_empa.cp2k.molecule_opt')
Cp2kSlabOptWorkChain = WorkflowFactory('nanotech_empa.cp2k.slab_opt')
Cp2kBulkOptWorkChain = WorkflowFactory('nanotech_empa.cp2k.bulk_opt')

In [ ]:
# Structure selector.
empa_viewer = EmpaStructureViewer()
build_slab = BuildSlab(title='Build slab')
dlink((empa_viewer, 'details'), (build_slab, 'details'))
dlink((empa_viewer, 'structure'), (build_slab, 'molecule'))

structure_selector = awb.StructureManagerWidget(
    viewer=empa_viewer,
    importers=[
        awb.StructureUploadWidget(title="Import from computer"),
        awb.StructureBrowserWidget(title="AiiDA database"),
        awb.SmilesWidget(title="From SMILES"),
        CdxmlUpload2GnrWidget(title="CDXML"),
    ],
    editors = [
        awb.BasicStructureEditor(title="Edit structure"),
        build_slab
    ],
    storable=False, node_class='StructureData')
display(structure_selector)

# Code.
computer_code_dropdown = awb.CodeDropdown(input_plugin='cp2k')
input_details = InputDetails()
create_input = ipw.Button(description="Create Input")

In [ ]:
dlink((empa_viewer, 'details'), (input_details, 'details'))
dlink((computer_code_dropdown, 'selected_code'),(input_details, 'selected_code'))

def prepare_mol_opt():

    builder = Cp2kMoleculeOptWorkChain.get_builder()
    parameters = input_details.final_dictionary.copy()
    builder.code = computer_code_dropdown.selected_code
    builder.structure = structure_selector.structure_node
    builder.charge = Int(parameters['charge'])
    
    ase_geom =  structure_selector.structure        

    # spin guess
    mag_list = [ 0 for t in ase_geom ]
    if parameters['multiplicity']>0:
        for i in awb.utils.string_range_to_list(parameters['spin_u'])[0]:
            mag_list[i] = 1
        for i in awb.utils.string_range_to_list(parameters['spin_d'])[0]:
            mag_list[i] = -1 
        builder.multiplicity = Int(parameters['multiplicity'])
        
    builder.magnetization_per_site = List(list=mag_list)    
    builder.vdw = Bool(parameters['vdw_switch'])
    builder.walltime_seconds = Int(parameters['walltime'])
    builder.metadata.label = 'Cp2kMoleculeOptWorkChain'
    builder.metadata.description = parameters['description']
    if 'DEBUG' in parameters['description']:
        builder.debug = Bool(True)
    #builder.metadata.label = final_dict['workchain'] 

    return builder

def prepare_slab_opt():

    builder = Cp2kSlabOptWorkChain.get_builder()
    parameters = input_details.final_dictionary.copy()
    builder.code = computer_code_dropdown.selected_code
    builder.structure = structure_selector.structure_node
    builder.charge = Int(parameters['charge'])
    
    ase_geom =  structure_selector.structure        

    # spin guess
    mag_list = [ 0 for t in ase_geom ]
    if parameters['multiplicity']>0:
        for i in awb.utils.string_range_to_list(parameters['spin_u'])[0]:
            mag_list[i] = 1
        for i in awb.utils.string_range_to_list(parameters['spin_d'])[0]:
            mag_list[i] = -1 
        builder.multiplicity = Int(parameters['multiplicity'])
        
    builder.magnetization_per_site = List(list=mag_list)    
    builder.vdw = Bool(parameters['vdw_switch'])
    builder.fixed_atoms = Str(parameters['fixed_atoms'])
    builder.protocol = Str(parameters['protocol'])
    builder.max_nodes = Int(parameters['max_nodes'])
    builder.walltime_seconds = Int(parameters['walltime'])
    builder.metadata.label = 'Cp2kSlabOptWorkChain'
    builder.metadata.description = parameters['description']
    if 'DEBUG' in parameters['description']:
        builder.debug = Bool(True)
    #builder.metadata.label = final_dict['workchain'] 

    return builder

def prepare_bulk_opt():

    builder = Cp2kBulkOptWorkChain.get_builder()
    parameters = input_details.final_dictionary.copy()
    builder.code = computer_code_dropdown.selected_code
    builder.structure = structure_selector.structure_node
    
    
    # cell opt
    if parameters['opt_cell']:
        builder.cell_opt = Bool(True)
        builder.symmetry = Str(parameters['cell_sym'])
        builder.cell_freedom = Str(parameters['cell_free'])
    
    # fixed atoms only in bulk opt
    else:
        builder.fixed_atoms = Str(parameters['fixed_atoms'])
    
    builder.charge = Int(parameters['charge'])
    
    ase_geom =  structure_selector.structure        

    # spin guess
    mag_list = [ 0 for t in ase_geom ]
    if parameters['multiplicity']>0:
        for i in awb.utils.string_range_to_list(parameters['spin_u'])[0]:
            mag_list[i] = 1
        for i in awb.utils.string_range_to_list(parameters['spin_d'])[0]:
            mag_list[i] = -1 
        builder.multiplicity = Int(parameters['multiplicity'])
        
    builder.magnetization_per_site = List(list=mag_list)    
    builder.vdw = Bool(parameters['vdw_switch'])

    builder.max_nodes = Int(parameters['max_nodes'])
    builder.walltime_seconds = Int(parameters['walltime'])
    builder.metadata.label = 'Cp2kBulkOptWorkChain'
    builder.metadata.description = parameters['description']
    if 'DEBUG' in parameters['description']:
        builder.debug = Bool(True)
    #builder.metadata.label = final_dict['workchain'] 

    return builder

In [ ]:
def after_submission(_=None):   
    structure_selector.value = None
    
btn_submit_mol_opt = awb.SubmitButtonWidget(Cp2kMoleculeOptWorkChain, 
                                input_dictionary_function=prepare_mol_opt
                               )
btn_submit_mol_opt.on_submitted(after_submission)

btn_submit_slab_opt = awb.SubmitButtonWidget(Cp2kSlabOptWorkChain, 
                                input_dictionary_function=prepare_slab_opt
                               )
btn_submit_slab_opt.on_submitted(after_submission)

btn_submit_bulk_opt = awb.SubmitButtonWidget(Cp2kBulkOptWorkChain, 
                                input_dictionary_function=prepare_bulk_opt
                               )
btn_submit_bulk_opt.on_submitted(after_submission)

In [ ]:
output = ipw.Output()
def update_all(_=None):
    btn_submit_mol_opt.btn_submit.disabled=True 
    btn_submit_slab_opt.btn_submit.disabled=True
    btn_submit_bulk_opt.btn_submit.disabled=True

def create(_=None):
    if computer_code_dropdown.selected_code is None:
        msg = 'please select a computer'
        can_submit = False
    else:            
        can_submit, msg, details_dict = input_details.return_final_dictionary()
 
    with output:
        clear_output()
        if can_submit:
            print("Ready to submit a ",details_dict['workchain']," type of calculation" )
            if details_dict['workchain'] == 'Cp2kMoleculeOptWorkChain':
                btn_submit_mol_opt.btn_submit.disabled = not can_submit
                display(btn_submit_mol_opt)
            elif details_dict['workchain'] == 'Cp2kSlabOptWorkChain':
                btn_submit_slab_opt.btn_submit.disabled = not can_submit
                display(btn_submit_slab_opt)  
            elif details_dict['workchain'] == 'Cp2kBulkOptWorkChain':
                btn_submit_bulk_opt.btn_submit.disabled = not can_submit
                display(btn_submit_bulk_opt) 
            else:
                print('something went wrong')
        else:
            print(msg)

create_input.on_click(create)
input_details.observe(update_all, names='details')

In [ ]:
display(ipw.VBox([input_details,computer_code_dropdown,create_input]), output)